In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pandas numpy 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/98/af/7be05277859a7bc399da8ba68b88c96b27b48740b6cf49688899c6eb4176/pandas-2.3.3-cp39-cp39-win_amd64.whl (11.4 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/81/c4/34e93fe5f5429d7570ec1fa436f1986fb1f00c3e0f43a589fe2bbcd22c3f/pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5c/23/c7abc0ca0a1526a0774eca151daeb8de62ec457e77262b66b359c3c7679e/tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- -----

In [4]:
!pip install torch_geometric

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/03/9f/157e913626c1acfb3b19ce000b1a6e4e4fb177c0bc0ea0c67ca5bd714b5a/torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/62/a1/3d680cbfd5f4b8f15abc1d571870c5fc3e594bb582bc3b64ea099db13e56/jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/0f/15/5bf3b99495fb160b63f95972b81750f18f7f4e02ad051373b669d17d44f2/aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/fb/76/641ae371508676492379f16e2fa48f4e2c11741bd63c48be4b12a6b09cba/aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a7/c2/fe1e52489ae3122415c51f387e221dd07737

In [6]:
!pip install torch
from torch_geometric.datasets import Planetoid, Amazon, Reddit, WikiCS, Flickr, WebKB, Actor, PolBlogs, CitationFull
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.transforms import AddSelfLoops
from torch_geometric.utils import subgraph, k_hop_subgraph
import torch_geometric.transforms as T
import torch
import math
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch_geometric.nn import global_add_pool, global_max_pool, GlobalAttention, global_mean_pool
from torch_geometric.nn.inits import glorot




edge_index = torch.tensor([[0, 1, 0],
                           [1, 2, 0]], dtype=torch.long)
x = torch.tensor([[1,2,3],
                 [4,5,6],
                 [6,7,8]])
y = torch.tensor([0,1,2])
data = Data(x=x, edge_index=edge_index, y=y)
# 创建 AddSelfLoops 转换
add_self_loops = AddSelfLoops()
# 应用转换
data = add_self_loops(data)
print(data.edge_index)
data.edge_index = torch.unique(data.edge_index, dim=1)
print(data.edge_index)

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/b9/dc/1f1f621afe15e3c496e1e8f94f8903f75f87e7d642d5a985e92210cc208d/torch-2.8.0-cp39-cp39-win_amd64.whl (241.2 MB)
tensor([[0, 1, 0, 0, 1, 2],
        [1, 2, 0, 0, 1, 2]])
tensor([[0, 0, 1, 1, 2],
        [0, 1, 1, 2, 2]])


In [8]:
%pwd

'c:\\Users\\11326\\Desktop\\MetaPrompt'

In [ ]:
!pip install scipy
def induced_graphs(data, device, smallest_size=10, largest_size=30):   # 构建诱导图的过程
    induced_graph_list = []
    from copy import deepcopy
    
    for index in range(data.x.size(0)):
        current_label = data.y[index].item()

        current_hop = 2
        subset, _, _, _ = k_hop_subgraph(node_idx=index, num_hops=current_hop,
                                            edge_index=data.edge_index, relabel_nodes=True)
        subset = subset

        while len(subset) < smallest_size and current_hop < 5:
            current_hop += 1
            subset, _, _, _ = k_hop_subgraph(node_idx=index, num_hops=current_hop,
                                                edge_index=data.edge_index)
            
        if len(subset) < smallest_size:
            need_node_num = smallest_size - len(subset)
            pos_nodes = torch.argwhere(data.y == int(current_label))   # Test data may leak
            pos_nodes = pos_nodes.to('cpu')
            subset = subset.to('cpu')
            candidate_nodes = torch.from_numpy(np.setdiff1d(pos_nodes.numpy(), subset.numpy()))
            candidate_nodes = candidate_nodes[torch.randperm(candidate_nodes.shape[0])][0:need_node_num]
            subset = torch.cat([torch.flatten(subset), torch.flatten(candidate_nodes)])

        if len(subset) > largest_size:
            subset = subset[torch.randperm(subset.shape[0])][0:largest_size - 1]
            subset = torch.unique(torch.cat([torch.LongTensor([index]).to(device), torch.flatten(subset).to(device)]))

        subset = subset.to(device)
        sub_edge_index, _ = subgraph(subset, data.edge_index, relabel_nodes=True)
        sub_edge_index = sub_edge_index.to(device)

        x = data.x[subset]

        induced_graph = Data(x=x, edge_index=sub_edge_index, y=data.y[index], index=index, node_idx=subset.cpu())
        add_self_loops = AddSelfLoops()
        induced_graph = add_self_loops(induced_graph)
        induced_graph.edge_index = torch.unique(induced_graph.edge_index, dim=1)
        induced_graph_list.append(induced_graph)
        if index%500 == 0:
            print(index)
    return induced_graph_list


transform_list = [T.AddSelfLoops(), T.ToUndirected(), T.NormalizeFeatures()]
transform = T.Compose(transform_list)
dataset = Planetoid(root='c:/Users/11326/Desktop/MetaPrompt', name='PubMed', transform=transform)
data = dataset[0]
graph_list = induced_graphs(data, 'cpu')
print(graph_list[0],graph_list[0].index)

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
     -- ------------------------------------- 3.4/46.2 MB 20.2 MB/s eta 0:00:03
     ----- ---------------------------------- 6.0/46.2 MB 15.4 MB/s eta 0:00:03
     ------- -------------------------------- 8.9/46.2 MB 14.6 MB/s eta 0:00:03
     -------- ------------------------------ 10.5/46.2 MB 12.8 MB/s eta 0:00:03
     ---------- ---------------------------- 12.1/46.2 MB 13.3 MB/s eta 0:00:03
     -------------- ------------------------ 17.3/46.2 MB 13.6 MB/s eta 0:00:03
     -------------- ------------------------ 17.6/46.2 MB 13.3 MB/s eta 0:00:03
     ----------------- --------------------- 21.2/46.2 MB 12.5 MB/s eta 0:00:02
     -------------------- ------------------ 24.1/46.2 MB 12.6 MB/s eta 0:00:02
     ----------------------- --------------- 27.5/46.2 MB 12.9 MB/s eta 0:00:02
     ------------------------- ------------- 30.1/46.2 MB 12.9 MB/s

Processing...
Done!


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
Data(x=[30, 500], edge_index=[2, 70], y=1, index=0) 0


In [11]:
def normalize_adj_tensor(adj):
    # gcn的归一化邻接矩阵方法
    D = torch.sum(adj, dim=1)
    D_inv = torch.pow(D, -1 / 2)
    D_inv[torch.isinf(D_inv)] = 0.
    D_mat_inv = torch.diag(D_inv)
    adj_norm = D_mat_inv @ adj @ D_mat_inv  # GCN的归一化方式
    return adj_norm

def edge_index_to_adjacency_matrix(edge_index, num_nodes, undirected=True, device='cpu'):  
    # 将edge_indedx转化为邻接矩阵
    # 构建一个大小为 (num_nodes, num_nodes) 的零矩阵  
    adjacency_matrix = torch.zeros(num_nodes, num_nodes, dtype=torch.uint8).to(device)
      
    # 使用索引广播机制，一次性将边索引映射到邻接矩阵的相应位置上  
    if undirected:
        adjacency_matrix[edge_index[0], edge_index[1]] = 1  
        adjacency_matrix[edge_index[1], edge_index[0]] = 1  
    else:
        adjacency_matrix[edge_index[0], edge_index[1]] = 1
    return adjacency_matrix


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
size_buffer = []
for graph in graph_list:
    size_buffer.append(graph.x.shape[0])
    adj = edge_index_to_adjacency_matrix(graph.edge_index, graph.x.shape[0])
    graph.adj = adj

print(graph_list[0])


Data(x=[30, 500], edge_index=[2, 70], y=1, index=0, adj=[30, 30])


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch

class GraphDataset(Dataset):
    def __init__(self, graph_list):
        self.graph_list = graph_list

    def __len__(self):
        return len(self.graph_list)

    def __getitem__(self, idx):
        return self.graph_list[idx]

class SequentialGraphLoader(DataLoader):
    def __init__(self, graph_list, batch_size=1):
        dataset = GraphDataset(graph_list)
        super(SequentialGraphLoader, self).__init__(dataset, batch_size=batch_size, shuffle=False)

    def __iter__(self):
        for i in range(0, len(self.dataset), self.batch_size):
            batch = self.dataset[i:i+self.batch_size]
            yield self.merge_graphs(batch)

    def merge_graphs(self, batch):
        # 合并子图的特征矩阵和邻接矩阵
        x_buffer = []
        adj_list = []
        y_buffer = []
        batch_sizes = []
        batch_indexs = []
        graph_indexs = []

        for i, graph in enumerate(batch):
            x = graph.x
            adj = graph.adj
            y = graph.y.unsqueeze(0)  # 确保 y 是二维张量
            x_buffer.append(x)
            adj_list.append(adj)
            y_buffer.append(y)
            batch_sizes.append(x.shape[0])  # 存储子图的大小
            batch_indexs = batch_indexs + [i] * x.shape[0]
            graph_indexs.append(graph.index)

        # 将特征矩阵堆叠
        x_combined = torch.cat(x_buffer, dim=0)  # 合并特征矩阵
        num_nodes = x_combined.size(0)  # 所有节点的总数
        
        # 构建大的邻接矩阵
        adj_combined = torch.zeros(num_nodes, num_nodes, device=x_combined.device)
        start = 0
        for i in range(len(batch)):
            end = start + batch_sizes[i]
            adj_combined[start:end, start:end] = adj_list[i]  # 填充对应的子图邻接矩阵
            start = end

        y_combined = torch.cat(y_buffer, dim=0)  # 合并标签
        batch_indexs = torch.tensor(batch_indexs)

        return x_combined, adj_combined, y_combined, batch_sizes, batch_indexs, graph_indexs

In [ ]:
from torch_geometric.data import Batch, Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
import math
import copy

modified_graph_list = copy.deepcopy(graph_list)
adj_changes = [torch.nn.Parameter(torch.FloatTensor(size, size)) for size in size_buffer] # 对每个子图的扰动
for adj_change in adj_changes:
    adj_change.data.fill_(0)
for modified_graph, graph, adj_change in zip(modified_graph_list, graph_list, adj_changes):
    change_square = adj_change - torch.diag(torch.diag(adj_change, 0))
    change_square = torch.clamp(change_square, -1, 1)
    modified_graph.adj = change_square + graph.adj

train_loader = SequentialGraphLoader(modified_graph_list[:800], batch_size=16) # 定义的Dataloader
test_loader = SequentialGraphLoader(modified_graph_list[800:], batch_size=16)
weights = []
w_velocities = []
hidden_sizes = [128 for i in range(2)]
previous_size = 500
out_dim = 7
for ix, nhid in enumerate(hidden_sizes):
    weight = torch.nn.Parameter(torch.FloatTensor(previous_size, nhid).to(device))
    w_velocity = torch.zeros(weight.shape).to(device)
    weights.append(weight)
    w_velocities.append(w_velocity)
    previous_size = nhid
    
output_weight = torch.nn.Parameter(torch.FloatTensor(previous_size, out_dim).to(device))
output_w_velocity = torch.zeros(output_weight.shape).to(device)
weights.append(output_weight)
w_velocities.append(output_w_velocity)
for w, v in zip(weights, w_velocities):
    stdv = 1. / math.sqrt(w.size(1))
    w.data.uniform_(-stdv, stdv)
    v.data.fill_(0)

criterion = nn.CrossEntropyLoss()  # 分类任务的损失函数


#初始化参数,避免梯度爆炸
for w, v in zip(weights, w_velocities):
    stdv = 1. / math.sqrt(w.size(1))
    w.data.uniform_(-stdv, stdv)
    v.data.fill_(0)


for epoch in range(10):
    print("epoch_num:", epoch)
    for x, adj, y, sizes, batch_indexs, graph_indexs in train_loader:
        loss = 0.0
        x, adj, y, batch_indexs = x.to(device), adj.to(device), y.to(device), batch_indexs.to(device)
        adj_norm = normalize_adj_tensor(adj)
        
        # 图卷积
        for i, w in enumerate(weights):
            if i != len(weights)-1:
                x = adj_norm @ x @ w
            else:
                x = global_mean_pool(x, batch_indexs) @ w
        
        # 计算损失
        loss = criterion(x, y)
        assert torch.isnan(loss).any()==False, "loss is NaN"
        if torch.isnan(loss).any():
            raise ValueError("loss is NaN!")
        weight_grads = torch.autograd.grad(loss, weights, create_graph=True)
        w_velocities = [0.9 * v + g for v, g in zip(w_velocities, weight_grads)]
        weights = [w - 0.01 * v for w, v in zip(weights, w_velocities)]
        
total_loss = 0.0
for x, adj, y, sizes, batch_indexs, graph_indexs in test_loader:
    x, adj, y, batch_indexs = x.to(device), adj.to(device), y.to(device), batch_indexs.to(device)
    adj_norm = normalize_adj_tensor(adj)
    
    # 图卷积
    for i, w in enumerate(weights):
        if i != len(weights)-1:
            x = adj_norm @ x @ w
        else:
            x = global_mean_pool(x, batch_indexs) @ w
        
    # 累计损失
    total_loss += criterion(x, y) * x.shape[0]

total_loss.backward(retain_graph=False)

# 提取元梯度
adj_grads = [adj_change.grad for adj_change in adj_changes]

# 验证形状
for grad, adj_change in zip(adj_grads, adj_changes):
    assert grad.shape == adj_change.shape
print("done")
print(adj_grads[0])
# -------------------------
# 在 total_loss.backward(...) 和 adj_grads 之后运行
# -------------------------

import numpy as np
import scipy.sparse as sp

# attack_ratio: 你在 adj_changes 定义处设定，比如 0.05 表示每个子图翻 / 注入 5% 的“可能边位”
# 若你还没定义，请在这里设置（或改为从上面单元读取）
attack_ratio = 0.50

# adj_grads 已经存在（list of tensors，shape 与对应 adj_change 相同）
# graph_list 与 modified_graph_list 在前面单元已定义
adj_grads = [g.detach().cpu().numpy() for g in adj_grads]  # 转 numpy 便于处理

def apply_budgeted_flips_to_subgraph(orig_adj, grad_mat, ratio):
    """
    orig_adj: torch tensor 或 numpy 二值邻接 (n_i, n_i)
    grad_mat: numpy ndarray, 对应 adj_change.grad 的绝对值敏感度矩阵 (n_i, n_i)
    ratio: 比例 (0~1)，表示要翻转的边/位置占上三角（不含对角）的比例
    返回: 修改后的二值邻接 numpy (n_i, n_i)
    """
    if isinstance(orig_adj, torch.Tensor):
        A = orig_adj.detach().cpu().numpy().astype(np.int8)
    else:
        A = np.array(orig_adj, dtype=np.int8)
    n = A.shape[0]
    # 只考虑上三角（i<j）的候选位置（无向图）
    iu = np.triu_indices(n, k=1)
    # score = abs(grad)
    scores = np.abs(grad_mat[iu])
    m = len(scores)
    k = max(1, int(np.round(ratio * m)))  # 至少选 1 个
    if k >= m:
        top_idx = np.arange(m)
    else:
        top_idx = np.argpartition(-scores, k-1)[:k]  # top-k indices within iu
    rows = iu[0][top_idx]
    cols = iu[1][top_idx]
    # flip edges at (rows, cols)
    A_new = A.copy()
    A_new[rows, cols] = 1 - A_new[rows, cols]  # flip: 1->0, 0->1
    A_new[cols, rows] = A_new[rows, cols]      # 对称
    # 保持对角为0
    np.fill_diagonal(A_new, 0)
    return A_new

# 遍历每个子图，按比例把梯度大的位置翻转成最终的二值邻接
for i, (sg, grad) in enumerate(zip(modified_graph_list, adj_grads)):
    # sg.adj 可能是 torch.Tensor；把 grad 传进去
    new_adj = apply_budgeted_flips_to_subgraph(sg.adj, grad, attack_ratio)
    # 把 numpy 转回 torch（或保持 numpy，后面拼接时统一处理）
    sg.adj = torch.from_numpy(new_adj).to(sg.x.device)

# 至此 modified_graph_list 已经包含“最终二值化后的子图邻接”
# 下一步：把子图拼回为一张大图并保存为 npz
# 需要每个 subgraph 包含原图 node id 映射：graph.index（你的 induced_graph 保存了 index 字段）
# 如果每个 subgraph 里有 attribute 'index' 为中心节点 id，可以用它和子图节点顺序做映射（若你在构造子图时保留了 node_idx 那更好）

# 这里基于你的 induced_graphs 函数：每个 subgraph 的 .index = 中心节点（单个node）
# 如果想要正确拼回原大图（non-overlapping partition），你必须在构造子图时保存 node_idx（原始节点 ids）。
# 假设你已经把 node_idx 存在每个 subgraph（名称为 'node_idx'），否则需要在划分时保存。

# 我这里实现一个 merge（非重叠子图或平均策略），先根据 node_idx 填 features 和 edges
def merge_subgraphs_to_full(modified_graph_list, num_nodes, feature_dim=None, threshold=0.5):
    # features
    # assume every sg.x shape = (n_i, d) and sg.node_idx exists
    d = feature_dim if feature_dim is not None else ensure_numpy(modified_graph_list[0].x).shape[1]
    feats = np.zeros((num_nodes, d), dtype=np.float32)
    feat_count = np.zeros((num_nodes,), dtype=np.int32)
    rows = []
    cols = []
    data = []
    labels = np.full((num_nodes,), -1, dtype=np.int64)
    for sg in modified_graph_list:
        if not hasattr(sg, 'node_idx'):
            raise RuntimeError("每个子图必须包含 node_idx 用于拼回原图。请在 induced_graphs 时保存 node_idx。")
        node_idx = ensure_numpy(sg.node_idx).astype(np.int64)
        x_np = ensure_numpy(sg.x)
        for local_i, orig_i in enumerate(node_idx):
            feats[orig_i] += x_np[local_i]
            feat_count[orig_i] += 1
        if hasattr(sg, 'y') and sg.y is not None:
            y_np = ensure_numpy(sg.y).reshape(-1)
                # 检查标签数量是否与节点数量匹配,
            if len(y_np) == len(node_idx):
                    # 标签数量与节点数量匹配，正常处理,
                    for local_i, orig_i in enumerate(node_idx):
                        labels[orig_i] = y_np[local_i]
            elif len(y_np) == 1:
                    # 只有一个标签，应用到所有节点（图级别标签）
                    label_value = y_np[0]
                    for orig_i in node_idx:
                        labels[orig_i] = label_value
            else:
                    # 标签数量不匹配，跳过或报警告
                    print(f"警告: 子图标签数量 {len(y_np)} 与节点数量 {len(node_idx)} 不匹配，跳过标签处理")
        # adjacency
        adj_local = sg.adj
        if isinstance(adj_local, torch.Tensor):
            adj_local = adj_local.detach().cpu().numpy()
        r_local, c_local = np.nonzero(adj_local)
        rows.extend(node_idx[r_local].tolist())
        cols.extend(node_idx[c_local].tolist())
        data.extend(adj_local[r_local, c_local].tolist())
    # average features where multiple assignments
    nonzero = feat_count > 0
    feats[nonzero] = feats[nonzero] / feat_count[nonzero][:, None]
    # build sparse adj
    adj_coo = sp.coo_matrix((np.array(data, dtype=np.float64), (np.array(rows), np.array(cols))),
                            shape=(num_nodes, num_nodes))
    adj_coo.setdiag(0)
    adj_coo = adj_coo.tocsr()
    adj_coo.eliminate_zeros()
    adj_coo = adj_coo.maximum(adj_coo.T)  # 保证对称
    # 二值化
    adj_bin = (adj_coo > threshold).astype(np.int8)
    return adj_bin.tocsr(), feats, labels

# helper ensure_numpy
def ensure_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.array(x)

# 这里 num_nodes 要设为原始大图节点数（例如 data.x.shape[0]）
num_nodes = data.x.shape[0]   # 你在 notebook 最开始加载的 data
adj_final, feats_final, labels_final = merge_subgraphs_to_full(modified_graph_list, num_nodes, feature_dim=data.x.shape[1], threshold=0.5)

# 保存为 npz（csr components + features + labels）
np.savez_compressed('modified_full_graph.npz',
                    adj_data=adj_final.data,
                    adj_indices=adj_final.indices,
                    adj_indptr=adj_final.indptr,
                    adj_shape=adj_final.shape,
                    features=feats_final.astype(np.float32),
                    labels=labels_final.astype(np.int64))
print("Saved modified_full_graph.npz")


# 同时保存为标准稀疏矩阵格式，兼容 DeepRobust
import scipy.sparse as sp
sp.save_npz('pubmed_mod_adj_005.npz', adj_final)
print("Saved pubmed_mod_adj_005.npz (sparse matrix format)")
print("Saved pubmed_mod_adj_005.npz (sparse matrix format)")
# 验证保存的文件
try:
    test_adj = sp.load_npz('pubmed_mod_adj_005.npz')
    print(f"验证成功: pubmed_mod_adj_005.npz - 形状: {test_adj.shape}, 类型: {type(test_adj)}")
except Exception as e:
    print(f"验证失败: {e}")


epoch_num: 0
epoch_num: 1
epoch_num: 2
epoch_num: 3
epoch_num: 4
epoch_num: 5
epoch_num: 6
epoch_num: 7
epoch_num: 8
epoch_num: 9
done
tensor([[ 0.0000,  0.1303,  0.1988, -0.0009, -0.2279,  0.0238,  0.3515,  0.1057,
          0.0676, -0.1496,  0.2897,  0.1671,  0.2784,  0.5615,  0.1402, -0.0403,
          0.3323, -0.0269,  0.1713, -0.0370, -0.0536,  0.0782, -0.3244,  0.1684,
          0.1267, -0.0102, -0.1639, -0.1298,  0.4705],
        [-0.1303,  0.0000,  0.0686, -0.1312, -0.3582, -0.1065,  0.2213, -0.0245,
         -0.0626, -0.2799,  0.1594,  0.0368,  0.1481,  0.4313,  0.0099, -0.1706,
          0.2021, -0.1572,  0.0410, -0.1673, -0.1839, -0.0520, -0.4547,  0.0381,
         -0.0036, -0.1405, -0.2941, -0.2601,  0.3403],
        [-0.1988, -0.0686,  0.0000, -0.1998, -0.4268, -0.1751,  0.1527, -0.0931,
         -0.1312, -0.3485,  0.0909, -0.0317,  0.0795,  0.3627, -0.0587, -0.2392,
          0.1335, -0.2258, -0.0276, -0.2359, -0.2525, -0.1206, -0.5233, -0.0305,
         -0.0721, -0.2090,

In [9]:
#在上述实验中可以发现需要较小的batch_size以支持meta-gradient的计算(e.g., batch_size=16); 如果太大则会出现Out-of-Memory, 因此在这里记录一下梯度图空间复杂度的理论分析：

#假设现在有m个子图，平均每个图有n个节点，那么对于一个epoch来说，有m/batch_size个batch;

#由于需要拼接整个batch的子图成一个大图，那么这个大图的邻接矩阵规模为 (n * batch_size)^2

#综上, 存储的梯度图理论空间复杂度为：
#O(m/batch_size*(n*batch_size)^2) = O(m*n^2*batch_size)

#进一步考虑到完整的surrogate_training有k个epoch,那么整个surrogate_training的空间复杂度为：
#O(k*m*n^2*batch_size)

#因此过大的batch_size必然导致过大的内存开销
#但如果batch_size过小, 比如极端情况下batch_size=1, 那么理论上讲导致surrogate_training过程过于“琐碎”, 无法感知到训练的全局方向。（TODO, 需要进一步做一个实验）
#上面问题更新:目前部分实验做下来batch_size不会特别影响surrogate_training的效果,但batch_size过小将导致更长的开销时间（TODO, 再进一步做一个实验）

#现在分析node_injection的理论空间复杂度:
#进一步假设注入的节点个数为原图个数的b%,那么一共有m*n*b%个注入节点,每个注入节点只考虑添加到一个子图,因此优化节点注入位置的拓扑有n条,那么需要元梯度的拓扑扰动总数为m*n^2*b%
#如果同时也考虑优化节点的d维特征,那么需要元梯度的特征总数为m*n*d*b%
#整个surrogate_training的空间复杂度为:
#O(k*m*n*(n+d)*b%)

#分析O(k*m*n^2*batch_size)和O(k*m*n*(n+d)*b%)大小, 即比较n*batch_size和（n+d）*b%
#往往为了不同任务域迁移,需要PCA降维,d不会过大，如100维
#因此显然 n*batch_size >>（n+d）*b%
